In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.cross_validation import  train_test_split
import time #helper libraries
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from numpy import newaxis

Using TensorFlow backend.


In [10]:
prices_dataset =  pd.read_csv('EURUSD1.csv', header=0)

In [13]:
yahoo_stock_prices = prices_dataset.Close.values.astype('float32')[0:1762]
yahoo_stock_prices = yahoo_stock_prices.reshape(1762, 1)
yahoo_stock_prices.shape

(1762, 1)

In [32]:
scaler = MinMaxScaler(feature_range=(0, 1))
yahoo_stock_prices = scaler.fit_transform(yahoo_stock_prices)

plt.plot(yahoo_stock_prices)
plt.show()

In [15]:
train_size = int(len(yahoo_stock_prices) * 0.80)
test_size = len(yahoo_stock_prices) - train_size
train, test = yahoo_stock_prices[0:train_size,:], yahoo_stock_prices[train_size:len(yahoo_stock_prices),:]
print(len(train), len(test))

(1409, 353)


In [17]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [23]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [24]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [25]:
#Step 2 Build Model
model = Sequential()

model.add(LSTM(input_dim=1, output_dim=50, return_sequences=True))

model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=False))

model.add(Dropout(0.2))

model.add(Dense(output_dim=1))

model.add(Activation('linear'))

start = time.time()

model.compile(loss='mse', optimizer='rmsprop')

print ('compilation time : ', time.time() - start)

('compilation time : ', 0.012001991271972656)


In [26]:
model.fit(trainX, trainY, batch_size=128, nb_epoch=10, validation_split=0.05)

Train on 1336 samples, validate on 71 samples
Epoch 1/10
1336/1336 [==============================] - 0s - loss: 0.1749 - val_loss: 0.3822 896/1336 [===================>..........] - ETA: 0s - loss: 0.1917
Epoch 2/10
1336/1336 [==============================] - 0s - loss: 0.0917 - val_loss: 0.1872
Epoch 3/10
1336/1336 [==============================] - 0s - loss: 0.0462 - val_loss: 0.0681
Epoch 4/10
1336/1336 [==============================] - 0s - loss: 0.0310 - val_loss: 0.0392
Epoch 5/10
1336/1336 [=============

In [40]:
#def plot_results_multiple(predicted_data, true_data,length):
#    plt.plot(scaler.inverse_transform(true_data.reshape(-1, 1))[length:])
#    plt.plot(scaler.inverse_transform(np.array(predicted_data).reshape(-1, 1))[length:])
#    plt.show()
    
##predict lenght consecutive values from a real one
#def predict_sequences_multiple(model, firstValue,length):
#    prediction_seqs = []
#    curr_frame = firstValue
    
#    for i in range(length): 
#        predicted = []        
        
#        print(model.predict(curr_frame[newaxis,:,:]))
#        predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
        
#        curr_frame = curr_frame[0:]
#        curr_frame = np.insert(curr_frame[0:], i+1, predicted[-1], axis=0)
        
#        prediction_seqs.append(predicted[-1])
        
#    return prediction_seqs

#predict_length=5
#predictions = predict_sequences_multiple(model, testX[0], predict_length)

#print(scaler.inverse_transform(np.array(predictions).reshape(-1, 1)))


#plot_results_multiple(predictions, testY, predict_length)

predicted = model.predict(testX)

source = np.reshape(testX, (len(testX), 1))

plt.plot(source[0:16],     color = 'green', label = 'test_x')
plt.plot(testY[0:16],     color = 'blue',  label = 'test_y')
plt.plot(predicted[0:16], color = 'red',   label = 'Predicted')

plt.xlabel('Time')
plt.ylabel('Stock Price')

plt.legend()
plt.show()
